# W261 Final Project - Experiment, Fine-Tune, Select the Optimal Pipeline
### Phase 3 leader: Dylan

## Section 02, Team 1
### Team Members

| <img src="https://avatars.githubusercontent.com/u/152460390?v=4" width="200"> | <img src="https://raw.githubusercontent.com/aimeewms/w261_final_project/main/images/Phase%201/Dylan.png" width="200"> | <img src="https://raw.githubusercontent.com/aimeewms/w261_final_project/main/images/Phase%201/Jiayue.jpg" width="200"> | <img src="https://raw.githubusercontent.com/aimeewms/w261_final_project/main/images/Phase%201/Vicky.jpg" width="200"> |
|---------------|------------------|--------------|----------------|
| Aimee Williams | Dylan Sivori | Jo (Jiayue) Zhu | Vicky Liu |
| aimeewms@berkeley.edu | dylan.sivori@berkeley.edu | jzhu868@berkeley.edu | vickylliu@berkeley.edu |


### **Abstract**

Flight delays pose significant challenges to airlines, airports, and passengers, leading to financial losses and operational inefficiencies. The objective of this project is to develop a machine learning algorithm that can predict whether a flight will be delayed (by at least 15 minutes) or not using airline and weather data. The dataset includes historical flight records, weather conditions, and air traffic data, sourced from the U.S. Department of Transportation and the National Oceanic and Atmospheric Administration (NOAA). Preliminary exploratory data analysis (EDA) will help identify key factors contributing to flight delays, such as adverse weather conditions, airport congestion, and carrier-specific patterns. Furthermore, our team had the choice to approach this problem through classification or regression, and we chose to approach this as a classification problem due to outliers in our dataset that might skew potential regression results.

While this type of predictive model could be valuable for many audiences, such as airline companies, we chose to focus on what would be best for flight passengers themselves. Our model could help passengers make informed decisions and reduce uncertainty when planning their trips. With this in mind, we chose F2-score as the primary evaluation metric for this project. Initially, we considered using recall as our main metric, since it prioritizes correctly flagging delayed flights even if it meant occasionally flagging on-time flights as delayed. This made sense from a passenger's perspective, as missing a delay could mean arriving unprepared to the airport and also being unprepared for disruptions to their itinerary, especially if they have connecting flights. However, we also recognized that a false alarm (incorrectly predicting a delay when the flight is actually on time) could be costly for our audience as well. For example, if a passenger relying on the model mistakenly believed their flight will be delayed, they might show up to the airport too late, potentially missing their flight. Using F2-score offers a compromise between these two costs by balancing recall and precision, which prioritizes minimizing false alarms. The equation for F2-score weights recall more heavily than precision, but does not ignore it entirely, ensuring that our model catches delays but in a way that is more practical and trustworthy for the audience.

For this phase of the project we have created more complex graph based features such as the origin and destination pageranks. We have also created both simple time based features such as binary indicators for if the day is a holiday (with a +/- two-day window) and if the day of the week is a weekend, as well as more complex time based features such as if a flight was previously delayed (with a two hour mask to avoid data leakage).

For our baseline model, we chose a simple Logistic model as well as a Gradient Boosted Tree model. We felt these models were simple enough to be improved upon (through added complexity, hyperparameter tuning as well as feature engineering) while still fitting the scope of the project. They both work well within the Classification problem framework and they both are easy to add complexity to. When training on 2015-2018 and testing on 2019, our F2 scores for our Logistic and Gradient Boosted Tree models (train, test) are (0.4436, 0.2975) and (0.6325, 	0.5088), respectively. We also tried an XGBoost Tree model which performed only marginally better (0.6634, 0.5357). Given the client ask, we also pursued training a multilayer percepton (MLP) Neural Network model. We experimented with different layers and saw a final best performance of (0.5522, 0.4227). Given all of these results we have found that our models are usually overfitting the training data. Our last attempt in this phase to control for this was to create an ensemble of all these models: one version where if any model predicted delay then the final prediction was a delay, and one version with majority prediction. Unfortunately we still saw the overfitting problem where tha first version reported (0.8317, 0.5337) and the majority prediction reported (0.6584, 0.5186) on training and test sets respectively. Based on these results, we have found the the best performing model is the XGBoost Tree based model with predictions being driven by weather based features, the engineered origin pagerank feature, and the destination lattitude and longitude.

With these learnings we can see that we are reaching a plateau with the current data that we possess. Moving forward we would seek to add data to our dataset that could help control for this, as well as additional feature engineering on new features that that are highly predictive. We would like to test the model in a real-world setting and ultimately aim to release this as an opt-in feature for passengers to help improve their travel experience.



### **Data Overview**

The main dataset for this project, OTPW data, is constructed by integrating airline data from the U.S. Department of Transportation with weather data from the National Oceanic and Atmospheric Administration repository. To briefly touch on the construction of this dataset: we believe the airline data from DOT was joined with the weather data from NOAA via a SQL join. Because latitude and longitude exist in the original weather dataset but not in the airline data, it is likely that a third table containing the latitude and longitudes of all airports was used to calculate the distance between airports and their closest weather stations.

The complete dataset spans from 2015 to 2019, covering five full years of airline operations and corresponding weather conditions. The full dataset consists of 31,673,119 records across 214 columns. The OTPW dataset is originally in CSV format but is first converted to Parquet format to improve processing efficiency and optimize performance during downstream tasks. We maintain this raw Parquet version for reference and checkpoint the dataset after major stages of processing. These checkpoints include: a preprocessing checkpoint after handling missing values and imputations, a feature engineering checkpoint, a dimensionality reduction checkpoint, and a final checkpoint following the train/test split. This approach helps avoid redundant computations while maintaining reproducibility throughout the pipeline.

A preliminary Exploratory Data Analysis revealed that a significant portion of the dataset has missing values, with over 100 columns containing more than 50% missing data. To streamline further processing, the team will establish a threshold to remove columns with excessive missing values. Additionally, our uniqueness analysis indicates that approximately 70 columns have only one or zero unique values, making them redundant for analysis. These findings suggest that a substantial number of columns can be eliminated to optimize the dataset for modeling. To accomplish this, we adopted the following workflow:

<img src="https://raw.githubusercontent.com/aimeewms/w261_final_project/refs/heads/main/images/Phase%202/Workflow.png">


To evaluate our model as a classification problem, we focused on the `DepDel15` variable, which is a binary indicator where 1 signifies a delay greater than 15 minutes and 0 indicates on-time or early departures. Upon examining the dataset, we found that a significant portion of the `DepDel15` column had missing values, primarily due to canceled flights. These canceled flights were excluded from the analysis, as they fall outside the scope of the prediction task. Furthermore, an analysis of the data revealed a class imbalance, with on-time flights significantly outnumbering delayed flights. This imbalance is especially pronounced in the five-year dataset, with on-time flights outnumbering delayed flights by approximately 4:1. As a result, special attention will be given to addressing this imbalance during model training to ensure robust performance.


The dataset will be split into training (2015-2018) and test (2019) sets in temporal order to prevent data leakage, ensuring that future data is never used to inform past predictions. To standardize numerical features, normalization will be performed only on the training set, and the same transformation will be applied to the validation and test sets. This ensures that information from the validation or test sets does not influence model training. For validation, blocked cross-validation will be used to eliminate data leakage. In this approach, the dataset will be divided into sequential blocks. Within each block, the data will be further split into training (80%) and validation (20%) sets in temporal order.

### Data Pre-Processing

For data cleaning and preprocessing, we employed a series of steps to handle missing values, ensure data consistency, and prepare the dataset for modeling.

We started by forward filling the hourly weather data with the most recent available value per station, provided that the most recent value was within the last 6 hours. If null values persisted after this step, we attempted to fill them with the average value for a given Origin City Market ID during the time period of the training dataset, ensuring no data leakage. Any remaining null values were imputed with the column average from the training dataset.

Next, we addressed the `HourlyPrecipitation` column, where "T" values (indicating trace amounts of precipitation) were replaced with a value of 0.001. This was chosen because it was smaller than the minimum value in the column but still greater than 0, accurately reflecting that precipitation had occurred. We also used regular expressions to remove any remaining alphabetic characters in this column.

For the `HourlyPressureChange` column, we removed the "+" symbol to allow proper casting to a numeric data type. Additionally, we filled null values in the `HourlyPressureTendency` column with -1, marking those entries as missing and ensuring they were distinguishable during analysis.

In the final stages of preprocessing, we took the following actions: dropped columns with over 50% missing values, as we were not comfortable imputing such a high proportion of missing data; removed redundant columns that exhibited a 1:1 relationship with others, thereby reducing the dimensionality of the dataset; eliminated any columns that could lead to data leakage, such as those containing information that would not be available at the time of prediction; and cast the remaining columns to their correct data types to ensure proper handling during model training and evaluation. **Figure 1** below provides a visual comparison of the null counts before and after these preprocessing steps.


**Figure 1 - Null Count Before and After Dropping Columns with 50% Missing Values (5 Year)**

<img src="https://raw.githubusercontent.com/vickylliu/W261/main/Null.png">


These  pre-processing steps helped to ensure a clean, consistent, and well-prepared dataset for subsequent modeling tasks.


### Feature Engineering

#### 1. Previous Flight Delay Indicator

The first feature, `prev_flight_delay_ind`, aimed to capture the delay propagation phenomenon in commercial aviation. We developed a binary indicator that flags whether the aircraft's previous flight was delayed, using tail number and flight chronology to track aircraft history. This feature directly addresses a common operational issue: when one flight is delayed, the same aircraft is often late for its next assignment due to constrained turnaround times.

##### ✈️ Impact of Previous Flight Delay on Current Flight Delays

| **Previous Flight Delayed** | **On-Time Flights** | **Delayed Flights** |
|-----------------------------|---------------------|---------------------|
| No (0)                      | 87.84 %             | 12.16 %             |
| Yes (1)                     | 68.61 %             | 31.39 %             |


As shown in **Figure 2**, it clearly revealed a sharp increase in the likelihood of delay when a previous flight was also delayed. On average, the probability of delay jumped by over 20% in such cases. From a modeling standpoint, this feature emerged as one of the most influential in boosting performance. In our experiments, models trained with this binary feature consistently outperformed those without, underscoring its strong predictive value for cascading delays within the network.

**Figure 2 - Proportion of Delays by Previous Flight Indicator (5 Years)**

<img src="https://raw.githubusercontent.com/vickylliu/W261/main/FIgure%20X%20-%20FE1%20Previous%20FLight%20INdicator.png">


#### 2. Holiday Indicator
The second engineered feature, `is_holiday`, introduced temporal seasonality by flagging holiday periods, with the aim of capturing the operational strains and travel surges commonly associated with public holidays. We constructed this indicator using statutory holiday calendars from both the U.S. and Canada for the years 2015 through 2019. To better model the extended travel rush, each holiday was expanded into a ±2-day window, creating a five-day span around each holiday where delays were most likely to occur.

Exploratory data analysis in **Figure 3** revealed that, despite expectations of higher delays during peak travel periods, the holiday indicator did not always show a significant increase in delay rates. In fact, flights scheduled during these holiday windows had a slightly lower average delay rate (17.6%) compared to non-holiday periods (18.3%). This suggests that airlines may anticipate higher demand and proactively adjust schedules, potentially improving on-time performance during peak times.

##### 📅 Holiday Indicator Summary

| **Is Holiday**  | **Delay Rate** | **Avg Delay Time (in minutes)** |
|----------------|----------------------|----------------|
| Yes (1)           | 17.59%         | 12.93                    |
| No (0)          | 18.26%         | 12.46                    |


**Figure 3 - Impact of Holiday Indicator on Flight Delays: Average Delay Time and Delay Rate (2015–2019)**

<img src="https://github.com/vickylliu/W261/blob/main/EDA2fe.JPG?raw=true">


Despite its marginal standalone impact on model performance, the holiday indicator enriched the model when used in combination with other temporal and network features. It provided valuable context that helped differentiate between routine operational fluctuations and predictable surges, allowing for more informed predictions under seasonal travel conditions.


#### 3. Weekend Indicator

The weekend indicator, `is_weekend`,  was another engineered feature aimed at capturing the distinct operational dynamics and travel behavior that often occur during weekends. Weekends tend to exhibit different travel patterns due to increased leisure travel, shorter work schedules, and higher vacation activity. The weekend indicator was defined by flagging flights that occurred on Saturday and Sunday, helping to model the variations in delays that may result from these specific travel days.

Exploratory data analysis revealed that weekend flights (Saturday and Sunday) experienced fewer delays compared to weekdays, as shown in **Figure 4**. This may be due to fewer operational flights during the weekend, better airline planning, and a reduction in business-related travel, which is typically more punctual and frequent during weekdays. The lower delay rates on weekends suggest that airlines may better allocate resources or schedule less congested flights during these times.

##### 🗓 Weekend Indicator Summary

| **Is Weekend**  | **Delay Rate** | **Avg Delay Time (in minutes)** |
|-----------------|----------------|---------------------------------|
| Yes (1)         | 17.4%          | 12.36                           |
| No (0)          | 18.5%          | 13.05                           |

**Figure 4 - Flight delay rates on weekends versus weekdays**

<img src="https://raw.githubusercontent.com/vickylliu/W261/main/delay%20vs%20non0delay%20%25.png">

While this feature had modest standalone impact on model accuracy, it proved useful in interaction with holiday and weather indicators. Together, these features gave the model stronger awareness of time-of-week and its indirect effects on delay dynamics.

#### 4. PageRank and Degree Centrality

Our structural delay analysis examines the average departure delay between origin-destination airport pairs over the 60-month dataset. As shown in **Figure 5**, the top 50 most delayed routes revealed consistently high delays for certain airport combinations—often involving major hubs or geographically complex locations. These patterns pointed to deeper network dynamics, suggesting that airport connectivity, congestion, and scheduling complexity play a significant role in driving persistent delays.

**Figure 5 - Average Departure Delay Between Origin and Destination (5 Years)**

<img src="https://raw.githubusercontent.com/vickylliu/W261/main/Average%20Departure%20Delay%20Between%20Origin%20and%20Destination%20(5%20Years).png">


To capture these structural characteristics, we engineered a set of network-based features using PageRank and Degree Centrality. Leveraging the GraphFrames library, we modeled the flight network as a directed graph where airports served as nodes and flights as weighted edges. The edge weights were derived from the total number of flights between airport pairs, which were then normalized to represent the probability of transitioning from one airport to another. This transformation enabled the graph algorithms to reflect not just connectivity, but traffic intensity and route prominence.

PageRank was used to determine the relative importance of airports within the flight network. Airports with higher PageRank scores are those that, despite potentially being farther away from specific destinations, are more "influential" in the network due to their connection to many other airports. In other words, these airports serve as major hubs where a significant number of flights converge. The higher the PageRank, the more congested and complex the operations at the airport, which likely leads to higher delay rates. We calculated the PageRank of airports using the weight of the edges (the number of flights between airports) as the measure of influence. This was done by calculating the normalized probability of each flight route (edge) based on its frequency within the network.

As illustrated in **Figure 6**, we observed a correlation between higher average PageRank scores and airports with more significant traffic flows. This supports the notion that the connectivity and operational complexity of an airport, as reflected by its PageRank, could influence delay patterns, as these airports often handle large volumes of flights.

**Figure 6 - Average PageRank by Origin Airport (5 Years)**

<img src="https://raw.githubusercontent.com/vickylliu/W261/main/FE4%20PageRank.png">


In parallel, we computed degree centrality to understand how extensively an airport was connected within the network. In-degree centrality represented the number of incoming connections to an airport—essentially, how many routes terminated there—while out-degree centrality captured the number of outbound connections. These values were normalized by the number of possible connections in the network to ensure comparability across airports. Airports with high out-degree centrality were often origin points for a wide array of routes and typically handled significant departure volumes, potentially increasing the risk of ground delays and congestion. High in-degree airports, conversely, dealt with high volumes of inbound traffic, which could strain gate availability and turnaround times.

In **Figure 7**, we visualized both centrality metrics to further validate their relationship with observed delays. Airports with high in- and out-degree centrality tended to exhibit more operational complexity, which may influence delay behavior.

**Figure 7 - Degree Centralities of Airports (5 Years)**

<img src="https://raw.githubusercontent.com/vickylliu/W261/main/FE4%20EDA%20Degree.png">

The origin and destination airports’ PageRank scores, as well as their normalized out-degree and in-degree centralities were designed to give the model a richer contextual understanding of each flight’s position within the broader network. By capturing both the importance and connectivity of origin and destination airports, we enabled the model to learn from structural bottlenecks and operational load patterns, thereby enhancing its ability to predict delays driven by systemic network effects.


##### 🧮 Added Graph Features

| **Feature Name**                  | **Description**                                                  |
|----------------------------------|------------------------------------------------------------------|
| `origin_pagerank`                | PageRank score of the origin airport                             |
| `dest_pagerank`                  | PageRank score of the destination airport                        |
| `origin_normalized_out_degree`   | Normalized out-degree centrality of the origin airport           |
| `dest_normalized_in_degree`      | Normalized in-degree centrality of the destination airport       |






### Post-processing EDA
After completing data preprocessing and feature engineering, we conducted an exploratory data analysis (EDA) to gain deeper insights into the processed features. A key aspect of this analysis was the examination of the average delay per origin airport. As shown in **Figure 8**, a geo-plot was used to visualize the average delay across different airports on a map. From this visualization, we observed notable regional patterns, with higher delay times concentrated in the Northeast region. This suggests that location-based factors, such as airport congestion and regional weather patterns, could significantly influence flight delays and are likely to be important features in improving model predictibility. 

**Figure 8 - Average Departure Delay for Each Origin Airport (5 Year)**

<img src="https://raw.githubusercontent.com/vickylliu/W261/main/newplot%20(1).png">

Further, we computed a Spearman correlation matrix for all numerical variables (**Figure 9**), as the distribution of these features is not consistently normal, and we are working with a classification problem rather than a regression problem. From the correlation matrix, we observed significant correlations among several geographical and weather-related features, indicating potential multicollinearity within the dataset. This insight will inform our next steps in dimensionality reduction prior to modeling.

**Figure 9 - Spearman Correlation Heatmap of Flight Delay & All Continuous Features (5 Year)**
<img src="https://raw.githubusercontent.com/vickylliu/W261/main/CM%20ALL.png">

To simplify the modeling process and enhance training efficiency, we performed dimensionality reduction by eliminating features that contribute minimally to the model. As noted earlier, we identified multicollinearity among certain numerical variables. To address this, we removed one feature from each pair with a correlation greater than 0.7 (**Figure 10**). Additionally, we applied LASSO regularization to further refine the feature space, discarding numerical features with coefficients below 0.001, as they had negligible impact. The table below (**Table 3**) lists the features removed using these two approaches.


**Figure 10 - Spearman Correlation Heatmap of Continuous Features (5 Year)**
<img src="https://raw.githubusercontent.com/vickylliu/W261/main/CM%20after.png">

**Table 3 - Summary of Feature Removal Based on Spearman Correlation and LASSO Regularization**

| Feature Removal Reason                 | Features Removed                                                                                                                                                               | Count |
|----------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|-------|
| High Correlation (> 0.7)               | `DEP_DELAY_DOUBLE`, `DEP_DELAY_NEW_DOUBLE`, `DEP_DELAY`, `DISTANCE`, `ELEVATION`, `HourlyAltimeterSetting`, `HourlyDewPointTemperature`, `HourlyWetBulbTemperature`, `origin_station_lat`, `origin_station_lon`, `origin_airport_lat`, `origin_airport_lon`, `dest_station_lat`, `dest_station_lon`, `dest_normalized_in_degree`, `origin_normalized_out_degree` | 16    |
| LASSO Regularization (< 0.001)         | `AIR_TIME`, `origin_station_dis`, `dest_station_dis`, `HourlyWindDirection`                                                                                                    | 4     |



### Feature Space
Following data pre-processing, correlation analysis, and dimensionality reduction using Lasso regularization, we finalized a refined set of 26 features for modeling. These include both continuous and categorical variables capturing key flight, airport, and weather information.

**Table 4 - Continuous Features in the Feature Space**
| Feature Name                | Description                                                                 |
|----------------------------|-----------------------------------------------------------------------------|
| `dest_airport_lat`         | Latitude of the destination airport, indicating its geographic location.    |
| `dest_airport_lon`         | Longtitude of the destination airport, indicating its geographic location.    |
| `HourlyDryBulbTemperature` | Ambient air temperature measured at the weather station.                    |
| `HourlyPrecipitation`      | Amount of precipitation recorded during the hour.                           |
| `HourlyPressureTendency`   | Change in atmospheric pressure over the previous three hours.               |
| `HourlyRelativeHumidity`   | Percentage of moisture in the air relative to the maximum possible.         |
| `HourlySeaLevelPressure`   | Atmospheric pressure adjusted to sea level, measured at the station.        |
| `HourlyStationPressure`    | Atmospheric pressure measured at the station’s actual elevation.            |
| `HourlyVisibility`         | Distance one can see horizontally, indicating weather clarity.              |
| `HourlyWindGustSpeed`      | Maximum wind speed observed over a short period during the hour.            |
| `HourlyWindSpeed`          | Average wind speed recorded during the hour.                                |
| `origin_pagerank`        | PageRank score (airport importance) of the origin airport.                |
| `dest_pagerank`        | PageRank score (airport importance) of the destination airport.


**Table 5 - Categorical Features in the Feature Space**
| Feature Name               | Description                                                                 |
|---------------------------|-----------------------------------------------------------------------------|
| `OP_CARRIER_AIRLINE_ID`   | Unique ID representing the operating airline.                               |
| `ORIGIN`                  | IATA code of the origin airport.                                            |
| `ORIGIN_STATE_ABR`        | Abbreviation of the origin airport’s state.                                 |
| `DEST`                    | IATA code of the destination airport.                                       |
| `DEST_STATE_ABR`          | Abbreviation of the destination airport’s state.                            |
| `DIVERTED`                | Indicator of whether the flight was diverted.                               |
| `DISTANCE_GROUP`          | Grouped category based on flight distance range.                            |
| `Month`          | Month                            |
| `origin_type`             | Type of the origin airport (e.g., large, medium, small hub).                |
| `dest_type`               | Type of the destination airport.                                            |
| `prev_flight_delay_ind`   | Indicator of whether the previous flight was delayed.                                 |
| `is_holiday`   | Holiday indicator for the flight scheduled departure date  |
| `is_weekend`   | Weekend indicator for the flight scheduled departure date                                |

###Train/Test Split

For modeling, the dataset spanning five years (2015–2019) contained 31,175,565 records in total. The first four years (2015–2018), containing 23,914,759 records, were used for training, while the year 2019, consisting of 7,260,806 records, was reserved as the blind test set and was never consulted during training.

A key challenge in this dataset is the significant class imbalance in the target variable `DEP_DEL15`, where on-time flights overwhelmingly outnumber delayed flights. This imbalance is clearly depicted in **Figure 11**, which shows the original distribution of classes before any balancing was applied. Such imbalance could result in a model that is biased towards predicting the majority class (on-time flights), potentially leading to poor predictive performance, especially for delayed flights.

**Figure 11 - Comparison of On Time vs Delayed Flights in Train and Test Sets (5 Years Data)**

<img src="https://raw.githubusercontent.com/vickylliu/W261/main/train%20test%20unbalanced.png">

To mitigate this issue, we applied downsampling to the training set to balance the distribution of on-time and delayed flights. After downsampling, the number of on-time flights was reduced to match the number of delayed flights, resulting in 4,311,870 delayed records and 4,311,847 on-time records in the training set (as shown in Table 6). This downsampling procedure was applied only to the training set, while the 2019 test set remained unchanged to provide an unbiased evaluation of the model's performance on real-world, imbalanced data. The data split and the resulting balanced distribution of on-time and delayed flights are illustrated in **Figure 12**.

**Figure 12 - Comparison of On Time vs Delayed Flights in Train and Test Sets (5 Years Data) After Downsampling**

<img src="https://raw.githubusercontent.com/vickylliu/W261/main/train%20test%20downsampled.png">

This downsampling approach helps reduce the bias toward the majority class, allowing the model to learn more effectively from both classes. As a result, we expect improved model performance and a more accurate evaluation of its predictive capabilities, with the test set providing an unbiased measure of how the model will perform on unseen data.

**Table 6 - Count of On Time and Delayed Flights in Train and Test Sets (Before and After Downsampling)**

| Dataset              | Delayed | On Time |
|----------------------|---------|---------|
| Test                 | 1,354,374 | 5,906,432 |
| Train (Before)       | 4,311,870 | 13,223,533|
| Train (After)        | 4,311,870 | 4,311,847 |


## **Data Leakage** 

In the context of flight delay prediction using machine learning, leakage occurs when the model is given access to information that would not be available at the time of prediction. For example, if predictions are made two hours before a flight's scheduled departure, only weather data available up to that point should be used. Including weather information from the two-hour window before departure—such as last-minute changes or conditions at departure time—would introduce leakage, as this data would not be accessible when the prediction is actually made.

To prevent leakage, our pipeline was carefully designed to respect the temporal structure of the flight data. During the data preprocessing stage when we imputed missing values in the weather data with any mean values (either first using the origin city market id and then the total column mean for remaining values), we made sure that those means were calculated using only training data. The null values that were imputed in both the training and test sets were filled with means calculated using the training data to ensure that there is no leakage.

During feature engineering, we took care to ensure that all derived features aligned with the temporal constraints of the prediction task, which requires predictions to be made two hours prior to scheduled departure. Features such as the holiday indicator (`is_holiday`) and weekend indicator (`is_weekend`) were deemed safe, as they are fully determined by the scheduled flight date and publicly available calendars. The `prev_flight_delay_ind` feature was carefully constructed using a lag function over `TAIL_NUM`, ordered by scheduled UTC departure times, and included a two-hour minimum gap to ensure only prior, observable delays were used—making it safe from leakage.

However, one critical source of leakage was identified in the construction of the PageRank feature, which was calculated using the full five years of flight data. This introduced unintended future information into the model, as PageRank captured airport connectivity based on flights that occurred after the time of prediction. For example, a flight in 2016 could be assigned a PageRank value influenced by airport activity in 2018 or 2019—information that would not have been available at prediction time. This violates the core principle of time-aware modeling and introduces future knowledge into past predictions. To correct this, the PageRank feature should instead be calculated on a rolling or yearly basis, where for each prediction timestamp, only flights that occurred prior to that date are included in the network. This adjustment could potentially ensure that airport connectivity scores remain historically accurate and that the model only learns from information that would have been available in real-time.

In the modeling stage, we implemented Blocked Time Series Split as our cross-validation strategy to minimize data leakage. Unlike standard Time Series Split, which expands the training set in each fold and can lead to overlapping between train and test sets, the blocked approach will avoid this risk. In a standard Time Series Split, the model sees the same early data in multiple training sets, which can cause data leakage — especially when using features that depend on time, like lag variables. By contrast, Blocked Time Series Split divides the data into equal-sized, non-overlapping chunks, so the training and test sets don’t overlap. This helps eliminate data leakage and ensures each test block truly represents future data the model hasn't seen during training.



## **Modeling Pipelines**


### Families of Input Features
The table below summarizes the 26 input features used in the models, grouped by feature family.

**Table 7 - Input Features by Family**
| Feature Family| Feature Names| Count |
|------------------------|-----------------------------------------------------------------------------------------------------------------------------------------|--------|
| **Weather-related**    | `HourlyDryBulbTemperature`, `HourlyPrecipitation`, `HourlyPressureTendency`, `HourlyRelativeHumidity`, `HourlySeaLevelPressure`, `HourlyStationPressure`, `HourlyVisibility`, `HourlyWindGustSpeed`, `HourlyWindSpeed` | 9      |
| **Location-based**     | `dest_airport_lat`, `dest_airport_lon`,`ORIGIN`, `ORIGIN_STATE_ABR`, `DEST`, `DEST_STATE_ABR`, `origin_type`, `dest_type`  | 8     |
| **Flight/Carrier-related**    | `OP_CARRIER_AIRLINE_ID`, `DISTANCE_GROUP`,`DIVERTED`, `prev_flight_delay_ind`       | 4      |
| **Graph-based**     | `origin_pagerank`, `dest_pagerank`   | 2    |
| **Time-based**    | `Month`,`is_holiday`,`is_weekend`    | 3      |


### Evaluation Metrics

| Metric        | Formula                                                                                     |
|---------------|---------------------------------------------------------------------------------------------|
| **Precision (P)**  | $$ P = \frac{TP}{TP + FP} $$                                                     |
| **Recall (R)**     | $$ R = \frac{TP}{TP + FN} $$                                                     |
| **F2 Score**       | $$ F_2 = \frac{(5 \cdot P \cdot R)}{(4 \cdot P + R)} $$                                  |

*Notes:*
- TP is the number of true positives.
- FP is the number of false positives.
- FN is the number of false negatives.

### Visualization of Modeling Pipelines
<img src="https://raw.githubusercontent.com/Dylan-Sivori/w261-FinalProject/main/Phase3_W261_ML_Pipeline.jpeg">

### Cluster Size Used For Model Training
The cluster details used for all model experiments below is 5-10 Workers, 160-320 GB Memory, 40-80 Cores; 1 Driver with 128 GB Memory, 32 Cores.

### Baseline Models

#### Model 1: Logistic Regression

The Logistic regression models works by applying a sigmoid function to the linear regression model, which effectively squashes values to fall between 0 and 1. We chose this Logistic Regression as our baseline because it is a simple model that is both easy to implement and easy to interpret. In the case of a binary classification problem such as the one we are working on, the formula for the logistic regression model where x is our inputs and w are the weights applied to the inputs is: $$\hat{y} = \frac{1}{ 1+e^{-(xw^T+b)}}$$

The loss function for the Logistic Regression is the Binary Cross-Entropy log loss which is defined as: 
$$
\mathcal{L} = -\frac{1}{N} \sum_{i=1}^{N} \left[ y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i) \right]
$$

For tuning the baseline model we can control the following hyperparameters: maxIter, ElasticNetParam, and RegParam. The maxIter parameter controls the number of iterations in training, the ElasticNetParam controls the balance between L1 and L2 regularization, and the RegParam to control the strength of the regularization.

##### Hyperparameter Tuning
To test whether or not hyperparameter tuning would affect our models' performance, we chose to implement a grid search on all our models. To accomplish this, we defined a grid of potential values to try for its key parameters, then applied a Cartesian product to generate all possible combinations of these values. We then iterate through each hyperparameter set, performing blocked time-based cross validation to track performance of each model on our chosen metric, F2-score. The best performing set of parameters was (`maxIter: 5, regParam: 0.1, elasticNetParam: 0.25`), but the difference between all experiments was minimal. Below are our results. 

**Table 8 - Logistic Hyperparameter Experiments**
| # | Model Parameters | Blocked Time Split CV F2 Score|
|----|------------------|--------------------------------|
| 1 | maxIter: 5, regParam: 0.01, elasticNetParam: 0 | 0.429738|
| 2 | maxIter: 5, regParam: 0.1, elasticNetParam: 0 | 0.426253|
| 3 | maxIter: 10, regParam: 0.01, elasticNetParam: 0 | 0.429556|
| 4 | maxIter: 10, regParam: 0.1, elasticNetParam: 0 | 0.426079|
| 5 | maxIter: 5, regParam: 0.01, elasticNetParam: 0.25 | 0.434784|
| 6 | **maxIter: 5, regParam: 0.1, elasticNetParam: 0.25** | 0.445610|
| 7 | maxIter: 10, regParam: 0.01, elasticNetParam: 0.25 | 0.434037|
| 8 | maxIter: 10, regParam: 0.1, elasticNetParam: 0.25 | 0.445610|
| 9 | maxIter: 5, regParam: 0.01, elasticNetParam: 0.5 | 0.439267|
| 10 | maxIter: 5, regParam: 0.1, elasticNetParam: 0.5 | 0.445610|
| 11 | maxIter: 10, regParam: 0.01, elasticNetParam: 0.5 | 0.438911|
| 12 | maxIter: 10, regParam: 0.1, elasticNetParam: 0.5 |  0.445610|
| 13 | maxIter: 5, regParam: 0.01, elasticNetParam: 0.75 |  0.442473|
| 14 | maxIter: 5, regParam: 0.1, elasticNetParam: 0.75 |  0.445610|
| 15 | maxIter: 10, regParam: 0.01, elasticNetParam: 0.75 |  0.442365|
| 16 | maxIter: 10, regParam: 0.1, elasticNetParam: 0.75 |  0.445610|
| 17 | maxIter: 5, regParam: 0.01, elasticNetParam: 1 |  0.444352|
| 18 | maxIter: 5, regParam: 0.1, elasticNetParam: 1 |  0.445610|
| 19 | maxIter: 10, regParam: 0.01, elasticNetParam: 1 |  0.444273|
| 20 | maxIter: 10, regParam: 0.1, elasticNetParam: 1 |  0.445610|

Total time for the above experiment: 1hr47m

After the exhaustive grid search, we train the baseline model on the full training set and evaluate its performance on the test set.

**Table 9 - Logistic Regression trained on Full Dataset**
| Model Parameters    | Train Set| Test Set | 
|---------------------|-----------|---------|
| maxIter: 5, regParam: 0.1, elasticNetParam: 0.25 |  0.4436 | 0.2975|

The above training took ~2.1 minutes to complete.


### Ensemble Models


#### Model 2: Gradient Boosting Tree

Gradient Boosting Trees (GBT) is an ensemble learning technique where multiple decision trees are trained sequentially to form a strong predictive model. Each new tree is trained to predict the residuals or errors from the predictions made by the previous trees. This iterative approach allows the model to focus on the instances that were poorly predicted by previous iterations, leading to better overall performance. We chose this model for its ability to capture complex, non-linear relationships in the data, with flexibility and robust performance in classification tasks, making it ideal for predicting flight delays.

##### Loss Function
The Binary Cross-Entropy Loss (also known as Log Loss) for Grandient Boosting Tree is defined as:

$$
\mathcal{L} = -\frac{1}{N} \sum_{i=1}^{N} \left[ y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i) \right]
$$

This loss function is used to quantify the difference between the predicted probabilities and the actual outcomes. The goal is to minimize this value through the iterative learning process, where each new tree in the gradient boosting model tries to correct the errors (residuals) of the previous trees. This means the model is continually improving its ability to predict whether a flight will be delayed based on the features in the dataset.

By minimizing this loss function, the Gradient Boosted Trees algorithm seeks to make the predicted probability of a flight delay as close as possible to the true labels, ultimately improving the model's ability to correctly classify flights as either delayed or on time.


##### Hyperparameter Tuning
We also evaluated whether hyperparameter tuning could improve the performance of the GBT model. Using the same blocked time series cross-validation strategy as before, we iterated through various combinations of key hyperparameters, including maxDepth, maxBins, maxIter, and stepSize, to assess their impact on the F2-score. The best-performing configuration was `maxDepth=5, maxBins=64, maxIter=10, stepSize=0.1`, achieving an F2-score of 0.594874. Interestingly, the results indicated that there was no substantial performance gain across the different parameter sets. This suggests that the model's performance is relatively stable and less sensitive to changes in these hyperparameters. A summary of our results is presented below.

**Table 10 - GBT Hyperparameter Experiments**

| **#** | **Model Parameters**                                                      | **Blocked Time Split CV F2 Score** |
|------:|---------------------------------------------------------------------------|------------------------------------|
| 1     | `maxDepth: 5, maxBins: 32, maxIter: 10, stepSize: 0.01`                     | 0.582127                           |
| 2     | `maxDepth: 5, maxBins: 32, maxIter: 10, stepSize: 0.05`                     | 0.586138                           |
| 3     | `maxDepth: 5, maxBins: 32, maxIter: 20, stepSize: 0.01`                     | 0.589319                           |
| 4     | `maxDepth: 5, maxBins: 32, maxIter: 20, stepSize: 0.05`                     | 0.586177                           |
| 5     | `maxDepth: 5, maxBins: 64, maxIter: 10, stepSize: 0.01`                 | 0.594963                       |
| 6     | `maxDepth: 5, maxBins: 64, maxIter: 10, stepSize: 0.05`                     | 0.587230                           |
| 7     | `maxDepth: 5, maxBins: 64, maxIter: 20, stepSize: 0.01`                     | 0.594808                           |
| 8     | `maxDepth: 5, maxBins: 64, maxIter: 20, stepSize: 0.05`                     | 0.585544                           |
| 9     | **`maxDepth: 6, maxBins: 32, maxIter: 10, stepSize: 0.01`**           | **0.598683**                           |
| 10    | `maxDepth: 6, maxBins: 32, maxIter: 10, stepSize: 0.05`                     | 0.590107                           |
| 11    | `maxDepth: 6, maxBins: 32, maxIter: 20, stepSize: 0.01`                     | 0.592143                           |
| 12    | `maxDepth: 6, maxBins: 32, maxIter: 20, stepSize: 0.05`                     | 0.595961                           |
| 13    | `maxDepth: 6, maxBins: 64, maxIter: 10, stepSize: 0.01`                     | 0.593602                           |
| 14    | `maxDepth: 6, maxBins: 64, maxIter: 10, stepSize: 0.05`                     | 0.587332                           |
| 15    | `maxDepth: 6, maxBins: 64, maxIter: 20, stepSize: 0.01`                     | 0.588206                           |
| 16    | `maxDepth: 6, maxBins: 64, maxIter: 20, stepSize: 0.05`                     | 0.592678                           |


#### Model 3: XGBoost Tree
XGBoost is a powerful ensemble learning algorithm that builds upon the foundation of gradient boosting. Similar to standard gradient boosting, it trains each new decision tree to correct the residual errors of the preceding ensemble. In the context of our flight delay prediction task, the objective is to minimize the binary cross-entropy loss, which measures the discrepancy between predicted probabilities and actual binary outcomes. Additionally, XGBoost incorporates regularization techniques (L1 and L2 penalties) to improve generalization and reduce the risk of overfitting, making the model both accurate and robust. We selected XGBoost for its built-in regularization capabilities, which strengthen the training process and improve model stability across different datasets. Additionally, XGBoost is optimized for speed and efficiency, resulting in shorter training times compared to traditional gradient boosting methods — making it a great choice for large datasets and complex modeling tasks.


##### Loss Function
The Binary Cross-Entropy Loss (also known as Log Loss) for XGBoost Tree is defined as:

$$
\mathcal{L} = -\frac{1}{N} \sum_{i=1}^{N} \left[ y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i) \right] + \Omega(f)
$$

This loss function is similar to that used in standard gradient boosting trees but includes an additional regularization term, 
Ω(f), which helps control model complexity and reduce overfitting. Regularization can help the model generalize better to unseen data and enhances training stability. 

XGBoost typically incorporates two types of regularization:
- L1 Regularizaiton(Lasso): Simplifies the model by shrinking some feature weights to zero, effectively removing less important features and improving efficiency. 
- L2 Regularization(Ridge): Stabilizes the model by discouraging large weights, ensuring no single feature dominates the predictions.


##### Hyperparameter Tuning
We also explored whether hyperparameter tuning could enhance the performance of the XGBoost model. Using the same blocked time series cross-validation strategy as before, we evaluated a wider range of hyperparameter combinations—given that XGBoost offers more tunable parameters compared to the GBT model. Specifically, we iterated over values for `max_depth, learning_rate, reg_lambda, reg_alpha, n_estimators, gamma, min_child_weight`, while keeping `base_score=0.5, tree_method='hist', num_workers=5` fixed.

The best-performing configuration was:
`max_depth=6, learning_rate=0.1, reg_lambda=1, reg_alpha=0.1, n_estimators=200, gamma=0, min_child_weight=1`, which achieved an F2-score of 0.608218.
However, similar to our findings with the GBT model, the results showed limited performance variation across the different hyperparameter combinations. This suggests that the model’s performance is relatively robust to the changes in this parameter space. Due to the large number of experiments conducted, the table below presents the top 16 cross-validation results for comparison.

**Table 11 - XGBoost Tree Hyperparameter Experiments**
| **#** | **Model Parameters**  | **Blocked Time Split CV F2 Score** |
|------:|------------------------------------------------------------------------------------------------------------------|------------------------------------|
| 1     | **`max_depth: 8, learning_rate: 0.1, reg_lambda: 1, reg_alpha: 1, n_estimators: 200, gamma: 1, min_child_weight: 5`** | **0.609781**                           |
| 2     | `max_depth: 8, learning_rate: 0.1,  reg_lambda: 3, reg_alpha: 1, n_estimators: 200, gamma: 1, min_child_weight: 1`  | 0.609316                           |
| 3     | `max_depth: 8, learning_rate: 0.1, reg_lambda: 1, reg_alpha: 1, n_estimators: 100, gamma: 1, min_child_weight: 5` | 0.609077                           |
| 4     | `max_depth: 8, learning_rate: 0.1, reg_lambda: 1, reg_alpha: 1, n_estimators: 200, gamma: 1, min_child_weight: 1` | 0.608767                       |
| 5     | `max_depth: 8, learning_rate: 0.1, reg_lambda: 3, reg_alpha: 1, n_estimators: 100, gamma: 1, min_child_weight: 1` | 0.608722                           |
| 6     | `max_depth: 6, learning_rate: 0.1,  reg_lambda: 1, reg_alpha: 1, n_estimators: 200, gamma: 1, min_child_weight: 5`  | 0.608709                           |
| 7     | `max_depth: 6, learning_rate: 0.1, reg_lambda: 1, reg_alpha: 1, n_estimators: 200, gamma: 1, min_child_weight: 1` | 0.608312                           |
| 8     | `max_depth: 8, learning_rate: 0.1,  reg_lambda: 1, reg_alpha: 1, n_estimators: 100, gamma: 1, min_child_weight: 1`  | 0.607998                           |
| 9     | `max_depth: 8, learning_rate: 0.1,  reg_lambda: 3, reg_alpha: 1, n_estimators: 200, gamma: 1, min_child_weight: 5`  | 0.607947                          |
| 10    | `max_depth: 6, learning_rate: 0.1,  reg_lambda: 3, reg_alpha: 1, n_estimators: 200, gamma: 1, min_child_weight: 5`  | 0.607879                           |
| 11    | `max_depth: 5, learning_rate: 0.1,  reg_lambda: 5, reg_alpha: 1, n_estimators: 200, gamma: 1, min_child_weight: 3`  | 0.604392                           |
| 12    | `max_depth: 6, learning_rate: 0.1,  reg_lambda: 1, reg_alpha: 1, n_estimators: 100, gamma: 1, min_child_weight: 1`  | 0.605604                           |
| 13    | `max_depth: 5, learning_rate: 0.1, reg_lambda: 5, reg_alpha: 1, n_estimators: 200, gamma: 0, min_child_weight: 3` | 0.605525                           |
| 14    | `max_depth: 6, learning_rate: 0.1, reg_lambda: 1, reg_alpha: 1, n_estimators: 100, gamma: 1, min_child_weight: 5` | 0.605374                           |
| 15    | `max_depth: 6, learning_rate: 0.1, reg_lambda: 3, reg_alpha: 1, n_estimators: 100, gamma: 1, min_child_weight: 5` | 0.605031                           |
| 16    | `max_depth: 5, learning_rate: 0.1, reg_lambda: 5, reg_alpha: 0.1, n_estimators: 200, gamma: 0, min_child_weight: 3` | 0.605014                           |




### Neural Network

#### Model 4: Multilayer Perceptron (MLP)

A multi-layer perceptron (MLP) model is a type of feedforward neural network model that takes input data and moves it forward through a series of layers. The model consists of an input layer, one or more hidden layers, and an output layer, with each layer contains nodes, or neurons, that are densely connected to each other. This means that every neuron in one layer is connected to every neuron in the next. These dense connections, as well as the model's non-linear activation function (in this case, the sigmoid function), allow MLP models to be able to learn complex, nonlinear patterns in data. However, due to the large number of parameters associated with the model (with every neuron having a bias and every connection having a weight), these types of model tend to train much slower than the models previously mentioned.

The loss function for the Multilayer Perceptron is the Binary Cross-Entropy log loss which is defined as: 
$$
\mathcal{L} = -\frac{1}{N} \sum_{i=1}^{N} \left[ y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i) \right]
$$

##### Hyperparameter Tuning
Lastly, we performed a series of hyperparameter tuning experiments to improve the performance of the MLP model. Using the same blocked time series cross-validation strategy as before, we iterated through various combinations of key hyperparameters, including maxIter, layers, and blockSize, to assess their impact on the F2-score. maxIter controls the number of training iterations, or how long the model learns. Layers controls the model architecture, including the number of hidden layers and how many neurons are included in each layer. blockSize controls the number of samples in each batch, impacting training speed and the stability of the model. The best-performing configuration was `'layers': [511, 256, 128, 2], 'maxIter': 50, 'blockSize': 128`, achieving an F2-score of 0.59 in our series of experiments, but a score of 0.5522 when fitted to the entire training dataset and 0.4227 when fitted to the test dataset, indicating that the model was overfitting to the training dataset. A summary of our results is presented below (please note that the difference in input layer size in the later experiments is due to increasing the number of features).

**Table 12 - MLP Hyperparameter Experiments**

| # | Model Parameters | Blocked Time Split CV F2 Score|
|------------------------|-----------------------------------------------------------------------------------------------------------------------------------------|--------|
| 1 | 'layers': [511, 128, 2], 'maxIter': 50, 'blockSize': 64, 'seed': 42| 0.50 |
| 2 | 'layers': [511, 128, 2], 'maxIter': 50, 'blockSize': 128, 'seed': 42 | 0.55 |
| 3 | 'layers': [511, 128, 2], 'maxIter': 100, 'blockSize': 64, 'seed': 42 | 0.49 |
| 4 | 'layers': [511, 128, 2], 'maxIter': 100, 'blockSize': 128, 'seed': 42 | 0.50 |
| 5 | 'layers': [511, 128, 64, 2], 'maxIter': 50, 'blockSize': 64, 'seed': 42 | 0.57 |
| 6 | 'layers': [511, 128, 64, 2], 'maxIter': 50, 'blockSize': 128, 'seed': 42 | 0.54 |
| 7 | 'layers': [511, 128, 64, 2], 'maxIter': 100, 'blockSize': 64, 'seed': 42 | 0.51 |
| 8 | 'layers': [511, 128, 64, 2], 'maxIter': 100, 'blockSize': 128, 'seed': 42 | 0.50 |
| 9 | 'layers': [511, 256, 128, 2], 'maxIter': 50, 'blockSize': 64, 'seed': 42 | 0.54 |
| **10** | **'layers': [511, 256, 128, 2], 'maxIter': 50, 'blockSize': 128, 'seed': 42** | **0.59** |
| 11 | 'layers': [511, 256, 128, 2], 'maxIter': 100, 'blockSize': 64, 'seed': 42 | 0.51 |
| 12 | 'layers': [511, 256, 128, 2], 'maxIter': 100, 'blockSize': 128, 'seed': 42 | 0.51 |
| 13 | 'layers': [513, 256, 128, 2], 'maxIter': 5, 'blockSize': 1024, 'seed': 42 | 0.51 |
| 14 | 'layers': [513, 256, 128, 2], 'maxIter': 5, 'blockSize': 2048, 'seed': 42 | 0.51 |
| 15 | 'layers': [513, 256, 128, 2], 'maxIter': 10, 'blockSize': 1024, 'seed': 42 | 0.46 |
| 16 | 'layers': [513, 256, 128, 2], 'maxIter': 10, 'blockSize': 2048, 'seed': 42 | 0.46 |

### Ensemble of Models

In a final attempt to control for overfitting, we gathered predictions from the baseline logistic model, the XGBoost model, and the MLP model to use the ensemble of outputs to create a final prediction. This did not incur incremental amount of cost or time since models were already trained and it would take just seconds to make predictions, join predictions together, and create a new final prediction value.

The first version of our ensemble would make the prediction of a delay if any of the models in the ensemble predicted a delay. The second version of our ensemble was a majority prediction where the majority of the models had predict a delay for the final prediction of a delay. Unfortunately we did not see a reduction in the overfitting that we saw in the experiments above. The table below shows the results of these two ensemble methods.

**Table 13 - Ensemble Cross Validation Experiments**

|Ensemble Method|Blocked Time Split CV F2 Score|
|---------------|--------------|
|Any Prediction|0.8274|
|Majority Decision|0.6581|


### **Results and Discussions**

After performing hyperparameter tuning using Blocked Time Series Cross-Validation, we identified the optimal parameters for each model. These parameters were then used to retrain the models on the full training dataset, followed by evaluation on the held-out test set. **Table 14** summarizes the F2 scores achieved by each model on both the training and test datasets, providing a clear comparison of model performance. 

**Table 14 - Results Table**
| Model  | Blocked Time Split CV F2 Score | Train F2 Score | Test F2 Score | Model Build Time (mins) |
|---------------------|-------------------------------|----------------|---------------|--------------------------|
| Logistic Regression   |  	0.4456  |  0.4436  |  0.2975  |  2.1m  |
| Gradient Boosting Tree | 0.5987 | 0.6235  | 0.5088 | 13.4m   |
| **XGBoost**      | **0.6098**  |  **0.6634**   | **0.5357**  | **3.4m**   |
| Multilayer Perceptron (MLP) | 0.5678 | 0.5522  | 0.4227  | 1h11m  |
| Any Prediction Ensemble Method | 0.8274 | 0.8317  | 0.5337  | N/A (pre-trained models above used for prediction only) |
| Majority Prediction Ensemble Method | 0.6581 | 0.6585  | 0.5186  | N/A (pre-trained models above used for prediction only) |

The XGBoost model emerged as the best overall performer, achieving an F2 score of approximately 0.66 on the training set and 0.54 on the test set — the highest across all models and evaluation stages. **Figure 13** presents the confusion matrix for the final train and test set using the best-performing model. While the performance margin over Gradient Boosted Trees was relatively small, XGBoost’s faster training time, aided by built-in regularization, makes it more practical for real-world use on large datasets. The Multilayer Perceptron (MLP) also performed reasonably well, but its significantly longer training time and relatively higher sensitivity to hyperparameter tuning made it less ideal for this use case—especially given the scale of flight data and the need for an efficient model that can quickly adapt to fast-changing conditions. In contrast, the logistic regression model lagged behind, especially on the test set, likely due to its limited ability to model non-linear patterns.

**Figure 13 - Confusion Matrix of XGBoost (Train & Test set)**

<img src="https://raw.githubusercontent.com/Jozhu123/w261_Final_Project/main/XGB_CM.png">

Using the best model, we visualized the top features as shown in **Figure 14**. The Feature Importance Plot is based on the weight metric, which reflects how often a feature is used to split the data across all trees. As expected, a significant proportion of the top features (70%) are weather-related, emphasizing their substantial influence on the model's predictions. Additionally, the new feature origin_pagerank has also emerged as one of the top predictors, demonstrating that graph-based features significantly contribute to our model's predictions. Interestingly, the destination airport’s longitude and latitude also emerged as key predictors — a surprising yet insightful result.

**Figure 14 Feature Importance Plot Based on Best-Tuned Parameters of XGBoost**

<img src="https://raw.githubusercontent.com/Jozhu123/w261_Final_Project/main/feature_XGB.png">

The noticeable decrease in F2 score from training to test across all models suggests that overfitting remains a concern, despite efforts such as cross-validation, leakage control, and regularization. Part of this performance gap may be attributed to the class imbalance present in the test set — while the models were trained and validated on balanced data, the real-world test distribution reflects a more skewed class ratio. In addition, challenges such as modeling high-cardinality categorical features and capturing complex temporal patterns may also contribute to reduced generalization performance. Increasing model complexity through hyperparameter tuning — such as deeper trees or more neurons per layer in neural networks — can further increase the risk of overfitting if not properly controlled.


##### Gap Analysis of the Best Pipeline
The gap analysis of the best-performing pipeline provides a comprehensive evaluation of the model's strengths and areas for improvement. The pipeline, which utilizes ensemble tree models as the leading model, outperforms the other models, including logistic regression and neural network models, in terms of F2 score—the key metric for this project.

The best-performing pipeline, utilizes FeatureHasher for categorical encoding. This approach is particularly advantageous as it is both memory-efficient and scalable, especially when compared to the OneHotEncoder used in the logistic regression model. The use of FeatureHasher enables the model to handle a large number of categories without a significant increase in memory consumption, making it suitable for datasets with high-cardinality categorical features. Additionally, the model was trained using Blocked Time Series Cross-Validation, which preserves temporal order and minimizes data leakage — making it better suited for real-world applications. 

While the pipeline is robust, there remain opportunities for further improvement, particularly in the area of feature engineering. Additional transformations could help the models capture more nuanced patterns in the data. For example, applying cyclical encoding to time-related features (such as hour-of-day and day-of-week) would better reflect their periodic nature. Moreover, incorporating rolling statistics or airport-level congestion metrics—such as the rolling delay rate over the past three hours at each airport—could enrich the temporal context and potentially improve the model’s predictive accuracy by providing short-term historical trends relevant to delay outcomes. Additionally, since the top features of our best model are primarily weather-related and graph-based, we will continue to focus on enhancing these feature families—either through advanced feature transformations or by improving the processing and parsing of weather data.  

Class imbalance, specifically between delayed and non-delayed flights, was addressed through downsampling of the majority class to create a balanced training set. While this approach improves learning during training, it can discard valuable information and lead to overly optimistic performance that does not generalize well to the naturally imbalanced test data. This discrepancy contributes to the observed drop in F2 score from training to test. To address this gap, future iterations should explore alternative approaches such as class weighting or threshold tuning within ensemble tree models to better reflect real-world distributions and improve minority class recall.

Although the ensemble tree models perform well, there are still opportunities for improvement. One area we have started exploring is ensemble-of-ensemble techniques — combining multiple diverse models such as XGBoost, MLP, and logistic regression to leverage their complementary strengths. This approach has the potential to enhance both robustness and predictive accuracy, particularly in complex or imbalanced scenarios. However, due to time constraints, the preliminary results from this approach have not yet outperformed our current best model. Further tuning and experimentation will be needed to fully evaluate its potential.

Feature selection and dimensionality reduction could also be optimized. While correlation filtering and LASSO have been applied, additional steps in feature importance analysis could help identify the most significant features, allowing the model to focus on the most relevant data. Principal Component Analysis (PCA) could be considered to reduce the dimensionality of highly correlated features, improving both computational efficiency and model generalization.


In summary, while the ensemble tree models provide the best performance in terms of the F2 score, there remain opportunities to improve other models, such as logistic regression and neural networks, by refining feature engineering, addressing class imbalance more effectively, and optimizing feature selection. These efforts would lead to further improvements in the overall performance and utility of the model for flight delay prediction.



### **Conclusion**

To summarize, the focus of this project was to predict whether or not a flight will be delayed by greater than fifteen minutes to inform flight passengers of upcoming delays so they can plan their schedules accordingly. This is particularly important because the consumer’s time is the most valuable resource that they have available to them, and saving consumers’ time and money benefits unsatisfied customers as well as airline companies profits. We believe that a machine learning pipeline such as ours with engineered time series features can accurately predict a flight delay two hours prior to its departure, as we have already made great strides in successfully taking our OTPW dataset, exploring it for insights, creating new predictive features (such as previous flight delay and pagerank scores), selecting key top features (such as weather-related variables and airport location), and developing an optimized machine learning pipeline despite the class imbalance of the variable we are trying to predict. As for our next steps, our final XGBoost model could be improved through developing additional highly predictive features, reducing the overfitting of our model, training more advanced models for prediction, and finally integrating our predictions into a testing environment.

____________________________________________________________


### **Phase Leader Plan**  

| **Week**       | **Dates**        | **Phase**                                     | **Phase Leader** |
|---------------|-----------------|----------------------------------------------|--------------------|
| **Week 1**    | Mar 10 - Mar 16  | Project Plan: Define problem, datasets, joins, tasks, and metrics | **Vicky Liu**      |
| **Week 2**    | Mar 17 - Mar 26  | Data Preprocessing & Exploratory Data Analysis (EDA) | **Jo (Jiayue) Zhu**     |
| **Week 3**    | Mar 27 - Apr 6   | Baseline Model, Hyperparameter Tuning, Feature Engineering & Scalability | **Aimee Williams** |
| **Week 4**    | Apr 7 - Apr 13   | Model Selection, Fine-Tuning & Evaluation | **Dylan Sivori**   |
| **Final Tasks** | Apr 14 - Apr 19 | Final Report & Presentation | **All Team Members** |



### **Credit Assignment**  

| **Task**                                             | **Estimated Hours** | **Responsible Person** |
|----------------------------------------------------|-----------------|--------------------|
| **FP Phase 1: Project Plan (Mar 10 - Mar 16)** |  |  |
| Determine work plan and tasks for the project              | 2 hours           | **Vicky**      |
| Describe datasets, sources, and joins              | 6 hours           | **Dylan & Jo**   |
| Define project goals, evaluation metrics           | 4 hours           | **Aimee**     |
| Outline project pipeline & methodology            | 4 hours           | **Dylan & Vicky** |
| Draft project proposal                            | 4 hours           | **All Members**    |
| Submit project plan                               | 3 hours           | **Vicky**      |
| **FP Phase 2: EDA, Baseline Pipeline, Scalability & Efficiency (Mar 17 - Apr 6)** |  |  |
| Data cleaning & Preprocessing: handle missing values & outliers   | 10 hours           | **Dylan**     |
| Engineer new features & Transformations           | 4 hours           | **Aimee** |
| Perform Exploratory Data Analysis (EDA)           | 5 hours           | **Jo**   |
| Visualize data distributions & correlations   | 5 hours           | **Vicky**      |
| Dimensionality Reduction   | 4 hours           | **Vicky**      |
| Implement baseline model - Logistic Regression | 10 hours | **Dylan**   |
| Implement ensemble tree models - Gradient Boost Tree          |  12 hours           | **Vicky** |
| Implement ensemble tree models - XGBoost         |  12 hours           | **Jo** |
| Evaluate baseline model performance              | 3 hours           | **Vicky**      |
| Optimize data pipeline for scalability           | 3 hours           | **Jo**     |
| Implement distributed/parallel training          | 5 hours           | **Dylan**   |
| Optimize model scoring pipeline                  | 5 hours           | **Jo**     |
| Grid Search Hyperparameter Tuning                | 10 hours           | **Aimee**     |
| **Interim presentation & report**                |**Estimated Hours**| **Responsible Person**    |
|Project Abstract, Description, Results & Tuning| 4 hours | **Aimee**|
|Pre-processing, Logistic Pipeline & Conclusion | 4 hours | **Dylan**|
|Post-processing EDA, Feature Space &  Tree Pipeline | 4 hours | **Vicky**|
|Dimensionality Reduction, Train/Test Split & Tree Pipeline | 4 hours | **Jo**|
| **FP Phase 3: Model Selection, Fine-Tuning, & Final Report (Apr 7 - Apr 19)** |  |  |
| Data cleaning & Preprocessing on 5 years data: fix data leakage on mean imputation   | 7 hours           | **Dylan**     |
| Feature Engineering: Create time based and graph based features   | 10 hours           | **Vicky**     |
| EDA and Downsampling on 5 year data and new features   | 5 hours           | **Vicky**     |
| Traing MLP Neural Network with at least 2 different architechtures   | 10 hours           | **Aimee**     |
| Grid Search and Hyperparameter tuning baseline Logistic Regression Model on 5 years data  | 5 hours           | **Dylan**     |
| Grid Search and Hyperparameter tuning Tree based models (GBT and XGB) on 5 years data  | 15 hours           | **Jo**     |
| Grid Search and Hyperparameter tuning MLP Models on 5 years data  | 10 hours           | **Aimee**     |
| Create Ensemble of all three models (majority voting and any voting) for predictions  | 8 hours           | **Dylan**     |
| **Final presentation & report**                |**Estimated Hours**| **Responsible Person**    |
|Key Definitions, Project Abstract, Modeling Pipeline, Baseline Model | 4 hours | **Dylan**|
|Project Description, MLP Reporting | 4 hours | **Aimee**|
|Time & Graph Based Feature Engineering and Preprocessing Steps | 4 hours | **Vicky**|
|Tree Based Reporting & Conclusion | 4 hours | **Jo**|

### **Appendix**  
For reference, the code used in this project plan can be accessed here: 
- [**Phase 3 Data Preprocessing Code Notebook**](https://dbc-fae72cab-cf59.cloud.databricks.com/editor/notebooks/1536813829684365?o=4021782157704243)
- [**Phase 3 Baseline Code Notebook (Logistic)**](https://dbc-fae72cab-cf59.cloud.databricks.com/editor/notebooks/1536813829684263?o=4021782157704243)
- [**Phase 3 Baseline Code Notebook (Trees)**](https://dbc-fae72cab-cf59.cloud.databricks.com/editor/notebooks/1536813829684322?o=4021782157704243)
- [**Phase 3 Baseline Code Notebook (MLP)**](https://dbc-fae72cab-cf59.cloud.databricks.com/editor/notebooks/1536813829684492?o=4021782157704243)
- [**Phase 3 Baseline Code Notebook (Ensemble)**](https://dbc-fae72cab-cf59.cloud.databricks.com/editor/notebooks/3201478130151282?o=4021782157704243)